In [1]:
import fix_yahoo_finance as yf
import pandas as pd
import requests
import pymongo
import datetime 

In [2]:
# Extracted from https://www.nasdaq.com/screening/company-list.aspx
file_path = "Resources/companylist.csv"
df = pd.read_csv(file_path)
df = df[["Symbol","Name","LastSale","MarketCap","IPOyear","Sector","industry","Summary Quote"]]

# Create Dataframe for Technology
df = df.loc[df["Sector"]=="Technology"]
df = df.iloc[0:50,:]
start_date = "2004-05-03"
end_date = "2019-05-03"

# Review Data
df.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote
0,DDD,3D Systems Corporation,11.15,$1.3B,NaN,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/ddd
3,WUBA,58.com Inc.,72.45,$10.74B,2013.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/wuba
4,EGHT,8x8 Inc,24.21,$2.32B,NaN,Technology,EDP Services,https://www.nasdaq.com/symbol/eght
7,ATEN,"A10 Networks, Inc.",6.49,$487.94M,2014.0,Technology,Computer Communications Equipment,https://www.nasdaq.com/symbol/aten
10,AAN,"Aaron&#39;s, Inc.",58.75,$3.98B,NaN,Technology,Diversified Commercial Services,https://www.nasdaq.com/symbol/aan


In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
# Define database and collection
db = client.EcoParameters_db

db.StockPrice_Tech.drop()

collection = db.StockPrice_Tech

In [5]:
# List of tickers to download
tickers = list(df["Symbol"])

# Loop to get data
for ticker in tickers:

    data = yf.download(ticker, start=start_date, end=end_date)
    
    
    try:
        data = data.resample('QS').mean()
        data.reset_index(inplace=True)
        data["Ticker"] = ticker
        data = data[["Ticker","Date","Close","Volume"]]
        list_dict_prices_stock = []
        for item, row in data.iterrows():
            dict_row ={
                "Ticker" : row["Ticker"],
                "Date" : row["Date"].strftime('%Y-%m-%d'),
                "Close" : row["Close"],
                "Volume" : row["Volume"]
                }
            list_dict_prices_stock.append(dict_row)
        db.StockPrice_Tech.insert_many(list_dict_prices_stock)
    except Exception as e:
        print(e)

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*******************

In [6]:
list_price = list(db.StockPrice_Tech.find({"Ticker":"WUBA"}))
print(list_price)

[{'_id': ObjectId('5ccdd4f7144148193844fe9b'), 'Ticker': 'WUBA', 'Date': '2013-10-01', 'Close': 31.716904761904765, 'Volume': 1285471.4285714286}, {'_id': ObjectId('5ccdd4f7144148193844fe9c'), 'Ticker': 'WUBA', 'Date': '2014-01-01', 'Close': 42.1855737704918, 'Volume': 1054572.1311475409}, {'_id': ObjectId('5ccdd4f7144148193844fe9d'), 'Ticker': 'WUBA', 'Date': '2014-04-01', 'Close': 42.43650793650793, 'Volume': 1387584.126984127}, {'_id': ObjectId('5ccdd4f7144148193844fe9e'), 'Ticker': 'WUBA', 'Date': '2014-07-01', 'Close': 46.273906249999996, 'Volume': 1521800.0}, {'_id': ObjectId('5ccdd4f7144148193844fe9f'), 'Ticker': 'WUBA', 'Date': '2014-10-01', 'Close': 41.26015625, 'Volume': 837734.375}, {'_id': ObjectId('5ccdd4f7144148193844fea0'), 'Ticker': 'WUBA', 'Date': '2015-01-01', 'Close': 42.30557377049182, 'Volume': 685167.2131147541}, {'_id': ObjectId('5ccdd4f7144148193844fea1'), 'Ticker': 'WUBA', 'Date': '2015-04-01', 'Close': 72.64174603174607, 'Volume': 1638768.253968254}, {'_id': O

In [7]:
list_price = list(db.StockPrice_Tech.find())
print(list_price)

[{'_id': ObjectId('5ccdd4f7144148193844fe5e'), 'Ticker': 'DDD', 'Date': '2004-04-01', 'Close': 3.813414634146342, 'Volume': 33739.0243902439}, {'_id': ObjectId('5ccdd4f7144148193844fe5f'), 'Ticker': 'DDD', 'Date': '2004-07-01', 'Close': 3.80390625, 'Volume': 33145.3125}, {'_id': ObjectId('5ccdd4f7144148193844fe60'), 'Ticker': 'DDD', 'Date': '2004-10-01', 'Close': 4.934374999999999, 'Volume': 90787.5}, {'_id': ObjectId('5ccdd4f7144148193844fe61'), 'Ticker': 'DDD', 'Date': '2005-01-01', 'Close': 6.858524590163933, 'Volume': 182291.80327868852}, {'_id': ObjectId('5ccdd4f7144148193844fe62'), 'Ticker': 'DDD', 'Date': '2005-04-01', 'Close': 6.88640625, 'Volume': 226753.125}, {'_id': ObjectId('5ccdd4f7144148193844fe63'), 'Ticker': 'DDD', 'Date': '2005-07-01', 'Close': 7.478749999999998, 'Volume': 215620.3125}, {'_id': ObjectId('5ccdd4f7144148193844fe64'), 'Ticker': 'DDD', 'Date': '2005-10-01', 'Close': 6.2188888888888885, 'Volume': 177500.0}, {'_id': ObjectId('5ccdd4f7144148193844fe65'), 'Tic